In [3]:
# Import required libraries
import pandas as pd
import dash
from dash import html
from dash import dcc
from dash.dependencies import Input, Output
import plotly.express as px

# Read the airline data into pandas dataframe
spacex_df = pd.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv")
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

# Create a dash application
app = dash.Dash(__name__)

# Create an app layout
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),
# 1Dropdown de sitio de lanzamiento
            dcc.Dropdown(id='site-dropdown', options=[
            {'label': 'All Sites', 'value': 'ALL'},
            {'label': 'CCAFS LC-40', 'value': 'CCAFS LC-40'},
            {'label': 'CCAFS SLC-40', 'value': 'CCAFS SLC-40'},
            {'label': 'KSC LC-39A', 'value': 'KSC LC-39A'},
            {'label': 'VAFB SLC-4E', 'value': 'VAFB SLC-4E'}
             ],
            value='ALL',
            placeholder="Select a Launch Site here",
            searchable=True
            ),
            html.Br(),
# 2. Pie chart sitio seleccionado
            html.Div(dcc.Graph(id='success-pie-chart')),
            html.Br(),

            html.P("Payload range (Kg):"),
 # 3: slider 
                dcc.RangeSlider(
                id='payload_slider',
                min=0, 
                max=10000, 
                step=1000,
                marks={0: '0(Kg)',2500: '2500 (Kg)', 5000: '5000 (Kg)',7500: '7500 (Kg)', 10000: '10000 (Kg)'},
                value=[min_payload, max_payload]
                ),
#4. Scatter plot sitio vs carga util
            html.Div(dcc.Graph(id='success-payload-scatter-chart')),
             ])

# Add a callback function for `site-dropdown` as input, `success-pie-chart` as output
@app.callback(
    Output(component_id='success-pie-chart', component_property='figure'),
    Input(component_id='site-dropdown', component_property='value')
)
def get_pie_chart(entered_site):
    filtered_df = spacex_df
    if entered_site == 'ALL':
        fig = px.pie(filtered_df, values='class', 
                    names='Launch Site', 
                    title='Total Successful Launches by Site')
    else:
        filtered_df = spacex_df[spacex_df['Launch Site'] == entered_site]
        filtered_df = filtered_df.groupby(['Launch Site','class']).size().reset_index(name='class count')
        fig = px.pie(filtered_df, values='class count', 
                     names='class',
                     title=f'Total Sucessful Launches for Site {entered_site}')
    return fig

# Add a callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output
@app.callback(
    Output(component_id='success-payload-scatter-chart', component_property='figure'),
    [Input(component_id='site-dropdown', component_property='value'), 
    Input(component_id="payload_slider", component_property="value")]
)
def get_scatter_chart(entered_site, payload_range):
    filtered_df = spacex_df[(spacex_df['Payload Mass (kg)']>= payload_range[0]) &
                            (spacex_df['Payload Mass (kg)']<= payload_range[1])]
    if entered_site == 'ALL':
        fig = px.scatter(filtered_df, x='Payload Mass (kg)', y='class',
                         color='Booster Version Category',
                         title='Payload vs. Success for All Sites')
    else:
        filtered_df = filtered_df[filtered_df['Launch Site']== entered_site]
        fig = px.scatter(filtered_df, x='Payload Mass (kg)', y='class',
                         color='Booster Version Category',
                         title=f'Payload vs. Success for site {entered_site}')
    return fig

# Run the app
#if __name__ == '__main__':
#    app.run_server(debug=True)
if __name__ == '__main__':
    app.run(debug=True, port=8051)